## Regression Model with TF DenseFeatures

In [3]:
### Dependancies

In [1]:
#this should move to a create lifecycle config
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   

#this should move to a create lifecycle config
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/ehrsample2'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance
print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [2]:
#this should move to a create lifecycle config
#create S3 Bucket
bucket_name = 'ehrsample2' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [2]:
column_list = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num_label']

In [3]:
#this should move to a start lifecycle config (or should this be a processing job?)

#swiss data set
swiss_dataset_path = "s3://ehrsample2/processedswitzerland.txt"

#old code
#try:
#  urllib.request.urlretrieve ("s3://ehrsample2/processedswitzerland.csv", "processedswitzerland.csv.csv")
#  print('Success: downloaded swiss_setn.csv.')
#except Exception as e:
#  print('Data load error: ',e)

try:
  swiss_df = pd.read_csv(swiss_dataset_path,  names=column_list).replace('?', np.nan)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

#cleveland data set
cleveland_dataset_path = "s3://ehrsample2/processedcleveland.txt"

#old code
#try:
#  urllib.request.urlretrieve ("s3://ehrsample2/processedcleveland.csv", "processedcleveland.csv")
#  print('Success: downloaded cleveland_setn.csv.')
#except Exception as e:
#  print('Data load error: ',e)

try:
  cleveland_df = pd.read_csv(cleveland_dataset_path,  names=column_list).replace('?', np.nan)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: Data loaded into dataframe.
Success: Data loaded into dataframe.


In [4]:
swissdfsample = swiss_df.head(4)
print(swissdfsample)

   age  sex  cp trestbps  chol  fbs restecg thalach exang oldpeak slope   ca  \
0   32    1   1       95     0  NaN       0     127     0      .7     1  NaN   
1   34    1   4      115     0  NaN     NaN     154     0      .2     1  NaN   
2   35    1   4      NaN     0  NaN       0     130     1     NaN   NaN  NaN   
3   36    1   4      110     0  NaN       0     125     1       1     2  NaN   

  thal  num_label  
0  NaN          1  
1  NaN          1  
2    7          3  
3    6          1  


In [ ]:
#old code from pre-S3, can remove
#from https://archive.ics.uci.edu/ml/datasets/Heart+Disease
#swiss_dataset_path = "./data/heart_disease_data/processed_swiss.csv"
#swiss_df = pd.read_csv(swiss_dataset_path).replace('?', np.nan)

In [3]:
#old code from pre-S3, can remove
#cleveland_df = pd.read_csv("./data/heart_disease_data/processed.cleveland.txt",  names=column_list).replace('?', np.nan)

In [5]:
#combine both sources into one
combined_heart_df = pd.concat([swiss_df, cleveland_df])

In [6]:
len(combined_heart_df)

426

In [7]:
dfsample = combined_heart_df.head(4)
print(dfsample)

    age  sex   cp trestbps  chol  fbs restecg thalach exang oldpeak slope  \
0  32.0  1.0  1.0       95   0.0  NaN       0     127     0      .7     1   
1  34.0  1.0  4.0      115   0.0  NaN     NaN     154     0      .2     1   
2  35.0  1.0  4.0      NaN   0.0  NaN       0     130     1     NaN   NaN   
3  36.0  1.0  4.0      110   0.0  NaN       0     125     1       1     2   

    ca thal  num_label  
0  NaN  NaN          1  
1  NaN  NaN          1  
2  NaN    7          3  
3  NaN    6          1  


### Model Preparation
- This simple verson of the dataset has only a three features. 

In [8]:
selected_features = ['sex',  'age', 'trestbps', 'thalach' ]
bp_df = combined_heart_df[selected_features].replace({1:"male", 0:"female"})

In [9]:
# TEST how many rows with at least a single null value
sum(bp_df.apply(lambda x: sum(x.isnull().values), axis = 1)>0)

2

In [10]:
len(bp_df[bp_df['trestbps'].isnull()==True])

2

In [11]:
bp_df.head()

,sex,age,trestbps,thalach
0,male,32.0,95,127
1,male,34.0,115,154
2,male,35.0,NaN,130
3,male,36.0,110,125
4,female,38.0,105,166


In [12]:
#for simplicity will drop rows with null since predictor is null 
clean_df = bp_df.dropna()

In [13]:
clean_df.head()

,sex,age,trestbps,thalach
0,male,32.0,95,127
1,male,34.0,115,154
3,male,36.0,110,125
4,female,38.0,105,166
5,female,38.0,110,156


In [14]:
clean_df['trestbps'] = clean_df['trestbps'].astype(float)
clean_df['thalach'] = clean_df['thalach'].astype(float)

In [15]:
#adapted from https://www.tensorflow.org/tutorials/structured_data/feature_columns
def df_to_dataset(df, predictor,  batch_size=32):
    df = df.copy()
    labels = df.pop(predictor)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [16]:
#split 80 20 train test split - not ideal 
train_dataset = clean_df.sample(frac=0.8,random_state=0)
test_dataset = clean_df.drop(train_dataset.index)

In [17]:
PREDICTOR_FIELD = 'trestbps'
batch_size = 128
train_ds = df_to_dataset(train_dataset, PREDICTOR_FIELD, batch_size=batch_size)
test_ds = df_to_dataset(test_dataset, PREDICTOR_FIELD, batch_size=batch_size)

In [18]:
train_ds

<DatasetV1Adapter shapes: ({sex: (?,), age: (?,), thalach: (?,)}, (?,)), types: ({sex: tf.string, age: tf.float64, thalach: tf.float64}, tf.float64)>

In [19]:
# create TF numeric feature
tf_numeric_age_feature = tf.feature_column.numeric_column(key='age', default_value=0, dtype=tf.float64)

In [20]:
b_list = [ 0, 18, 25, 40, 55, 65, 80, 100]
#create TF bucket feature from numeric feature
tf_numeric_age_feature
tf_bucket_age_feature = tf.feature_column.bucketized_column(source_column=tf_numeric_age_feature, boundaries= b_list)

In [21]:
#using list b/c small number of unique values
gender_vocab = tf.feature_column.categorical_column_with_vocabulary_list(
      'sex', bp_df['sex'].unique())
gender_one_hot = tf.feature_column.indicator_column(gender_vocab)

In [22]:
# add cross features - use example from TF
crossed_age_gender_feature = tf.feature_column.crossed_column([tf_bucket_age_feature, gender_vocab], hash_bucket_size=1000)
tf_crossed_age_gender_feature = tf.feature_column.indicator_column(crossed_age_gender_feature)

In [23]:
feature_columns = [ tf_crossed_age_gender_feature, tf_bucket_age_feature, gender_one_hot ]
dense_feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [24]:
# Use same architecture as example
def build_model(dense_feature_layer):
  model = tf.keras.Sequential([
    dense_feature_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [25]:
model = build_model(dense_feature_layer)

In [26]:
EPOCHS = 1000
early_stop = tf.keras.callbacks.EarlyStopping(monitor='mse', patience=50)     
history = model.fit(train_ds,   callbacks=[early_stop], epochs=EPOCHS,  verbose=1)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 3 steps
Epoch 1/1000


FailedPreconditionError: Table not initialized.
	 [[{{node sequential/dense_features/age_bucketized_X_sex_indicator/hash_table_Lookup/LookupTableFindV2}}]]

In [ ]:
loss, mae, mse = model.evaluate(test_ds, verbose=2)
print("MAE:{}\nMSE:{}".format(mae, mse))

In [ ]:
test_labels = test_dataset[PREDICTOR_FIELD].values

In [ ]:
test_predictions = model.predict(test_ds).flatten()

In [ ]:
model_pred_outputs = {
    "pred": test_predictions,
    "actual_value": test_labels,
}
model_output_df = pd.DataFrame(model_pred_outputs)

In [ ]:
# Convert Regression Output to binary classification output
model_output_df.tail()

In [ ]:
#convert to binary prediction for Brier score - resting bps above 130 
def convert_to_binary(df, pred_field, actual_field):
    # score is the field name we will use for predictions and is what Aequitas uses
    df['score'] = df[pred_field].apply(lambda x: 1 if x>=130 else 0 )
    # label_value is the field name we will use for the truth value and is what Aequitas uses
    df['label_value'] = df[actual_field].apply(lambda x: 1 if x>=130 else 0)
    return df

In [ ]:
binary_df = convert_to_binary(model_output_df, 'pred', 'actual_value')
binary_df.head()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score

In [ ]:
y_true = binary_df['label_value'].values 
y_pred = binary_df['score'].values

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
roc_auc_score(y_true, y_pred)

## L4 - 2: Demographic Bias Analysis

### Instructions
- Using the Compas dataset prepared by Aequitas, perform a Fairness Disparity Analysis on the under 25 Asian female reference group. See the documentation for reference -https://github.com/dssg/aequitas. In particular focus your analysis on fairness and disparity for FPR and where applicable try to leverage some of the visualizations Aequitas provides.

### Aequitas Compas Dataset
- Using 2016 dataset from ProPublica for automated criminal risk assessment algorithms and adapted from Aequitas notebook - https://github.com/dssg/aequitas/blob/master/docs/source/examples/compas_demo.ipynb
- Preprocessed using this script --https://github.com/dssg/aequitas/blob/master/examples/compas_data_for_aequitas.py

The Aequitas Bias() class is used to calculate disparities between groups based on the crosstab returned by the Group() class get_crosstabs() method we used for preprocessing the dataset. 

### Instructions from Aequitas on Calculating Disparities across Reference Groups
(adapted from https://github.com/dssg/aequitas/blob/master/docs/source/examples/compas_demo.ipynb)

Disparities are calculated as a ratio of a metric for a group of interest compared to a base group. For example, the False Negative Rate Disparity for black defendants vis-a-vis whites is:$$Disparity_{FNR} =  \frac{FNR_{black}}{FNR_{white}}$$

Below, we use get_disparity_predefined_groups() which allows us to choose reference groups that clarify the output for the practitioner.

The Aequitas Bias() class includes two additional get disparity functions: get_disparity_major_group() and get_disparity_min_metric(), which automate base group selection based on sample majority (across each attribute) and minimum value for each calculated bias metric, respectively.

The get_disparity_predefined_groups() allows user to define a base group for each attribute.

Disparities Calculated Calcuated:
- True Positive Rate Disparity	'tpr_disprity'
- True Negative Rate	'tnr_disparity'
- False Omission Rate	'for_disparity'
- False Discovery Rate	'fdr_disparity'
- False Positive Rate	'fpr_disparity'
- False NegativeRate	'fnr_disparity'
- Negative Predictive Value	'npv_disparity'
- Precision Disparity	'precision_disparity'
- Predicted Positive Ratio$_k$ Disparity	'ppr_disparity'
- Predicted Positive Ratio$_g$ Disparity	'pprev_disparity'

**How do I interpret calculated disparity ratios?**
The calculated disparities from the dataframe returned by the Bias() class get_disparity_ methods are in relation to a reference group, which will always have a disparity of 1.0.

The differences in False Positive Rates, noted in the discussion of the Group() class above, are clarified using the disparity ratio (fpr_disparity). Black people are falsely identified as being high or medium risks 1.9 times the rate for white people.

As seen above, False Discovery Rates have much less disparity (fdr_disparity), or fraction of false postives over predicted positive in a group. As reference groups have disparity = 1 by design in Aequitas, the lower disparity is highlighted by the fdr_disparity value close to 1.0 (0.906) for the race attribute group 'African-American' when disparities are calculated using predefined base group 'Caucasian'. Note that COMPAS is calibrated to balance False Positive Rate and False Discovery Rates across groups.


**How does the selected reference group affect disparity calculations?**
Disparities calculated in the the Aequitas Bias() class based on the crosstab returned by the Group() class get_crosstabs() method can be derived using several different base gorups. In addition to using user-specified groups illustrated above, Aequitas can automate base group selection based on dataset characterisitcs:

Evaluating disparities calculated in relation to a different 'race' reference group
Changing even one attribute in the predefined groups will alter calculated disparities. When a different pre-defined group 'Hispanic' is used, we can see that Black people are 2.1 times more likely to be falsely identified as being high or medium risks as Hispanic people are (compared to 1.9 times more likely than white people), and even less likely to be falsely identified as low risk when compared to Hispanic people rather than white people.

In [ ]:
# Use Aequitas Data that was transformed 
compas_df = pd.read_csv("./data/compas_for_aequitas.csv")

In [ ]:
compas_df.head()

### Solution

### Dataset Preparation

In [ ]:
# Aequitas
from aequitas.preprocessing import preprocess_input_df
from aequitas.group import Group
from aequitas.plotting import Plot
from aequitas.bias import Bias
from aequitas.fairness import Fairness

ae_subset_df = compas_df[['sex', 'age_cat', 'race', 'score', 'label_value']]
ae_df, _ = preprocess_input_df(ae_subset_df)
g = Group()
xtab, _ = g.get_crosstabs(ae_df)
absolute_metrics = g.list_absolute_metrics(xtab)
clean_xtab = xtab.fillna(-1)
aqp = Plot()
b = Bias()

### Summarized Metric View

In [ ]:
absolute_metrics = g.list_absolute_metrics(xtab)
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

In [ ]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics].round(2)

In [ ]:
p = aqp.plot_group_metric_all(xtab, metrics=['tpr', 'fpr', 'ppr', 'pprev', 'fnr'], ncols=5)

### Add Reference Group

In [ ]:
bdf = b.get_disparity_predefined_groups(clean_xtab, 
                    original_df=ae_df, 
                    ref_groups_dict={'race':'Asian', 'sex':'Female', 'age_cat':'Less than 25'},
                    alpha=0.05, 
                    check_significance=False)

f = Fairness()
fdf = f.get_group_value_fairness(bdf)

### FPR Analysis

In [ ]:
 fpr_disparity = aqp.plot_disparity(bdf, group_metric='fpr_disparity', 
                                       attribute_name='race')

African Americans are over 5x more likely to be falsely identified as well as Hispanic and Caucasian have over 2x more likely to be falsely identified than Asian.

### Absolute Value Fairness Determination
- Red = False/Not Fair
- Green = True/Fair

In [ ]:
fpr_fairness = aqp.plot_fairness_group(fdf, group_metric='fpr', title=True)

## L4 - 3: Uncertainty Estimation Model

### Instructions
- Given the Swiss heart disease dataset we have been working with, create an uncertainty estimation model that accounts for Epistemic Uncertainty as well. Provide the mean and standard deviation outputs.
- https://blog.tensorflow.org/2019/03/regression-with-probabilistic-layers-in.html

In [ ]:
import tensorflow_probability as tfp

In [ ]:
'''
Adapted from Tensorflow Probability Regression tutorial  https://github.com/tensorflow/probability/blob/master/tensorflow_probability/examples/jupyter_notebooks/Probabilistic_Layers_Regression.ipynb    
'''
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(2*n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Independent(
            tfp.distributions.Normal(loc=t[..., :n],
                                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims=1)),
    ])


def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfp.layers.VariableLayer(n, dtype=dtype),
        tfp.layers.DistributionLambda(lambda t: tfp.distributions.Independent(
            tfp.distributions.Normal(loc=t, scale=1),
            reinterpreted_batch_ndims=1)),
    ])


In [ ]:
def build_seq_prob_model(feature_layer):
    model = tf.keras.Sequential([
        feature_layer,
        tf.keras.layers.Dense(150, activation='relu'),
        tf.keras.layers.Dense(75, activation='relu'),
           tfp.layers.DenseVariational(1+1, posterior_mean_field, prior_trainable),        
        tfp.layers.DistributionLambda(            
            lambda t:tfp.distributions.Normal(loc=t[..., :1],
                                            scale=1e-3 + tf.math.softplus(0.1 * t[...,1:])
                                             )
        ),
    ])
    return model

def build_prob_model(train_ds,   feature_layer,  epochs=5, loss_metric='mse'):
    model = build_seq_prob_model(feature_layer)
    negloglik = lambda y, rv_y: -rv_y.log_prob(y)
    loss = negloglik
    model.compile(optimizer='adam', loss=loss, metrics=[loss_metric])
    history = model.fit(train_ds, 
                        epochs=epochs)
    return model, history

In [ ]:
prob_model, history = build_prob_model(train_ds, dense_feature_layer,  epochs=1000)

In [ ]:
feature_list = ['sex', 'age', 'thalach']
x_tst = dict(test_dataset[feature_list])
yhat = prob_model(x_tst)
prob_preds = prob_model.predict(test_ds)
m = yhat.mean()
s = yhat.stddev()

In [ ]:
prob_outputs = {
    "pred": prob_preds.flatten(),
     "actual_value": test_dataset['trestbps'].values,
    "pred_mean": m.numpy().flatten(),
    "pred_std": s.numpy().flatten()
}
prob_output_df = pd.DataFrame(prob_outputs)

In [ ]:
prob_output_df.head()

In [ ]:
yhats = [prob_model(x_tst) for _ in range(10)]

In [ ]:
for i, yhat in enumerate(yhats):
    m = np.squeeze(yhat.mean())
    s = np.squeeze(yhat.stddev())
    print("Mean distributions sampled:{}".format(m))
    print("Standard deviation distributions sampled:{}".format(s))
